# Quick Metrics Computation

## Overview

This notebook provides a fast way to compute segmentation metrics (Dice, IoU) for a small subset of timepoints. It is useful for quick sanity checks during development.

## What This Notebook Does

1. Loads our RF predictions from `mask_santi/`
2. Generates baseline predictions (Otsu, Hysteresis) from raw Channel 3
3. Creates proxy ground truth from Channel 3 intensity
4. Computes Dice and IoU for each method

## When to Use

- Quick validation during development
- Sanity check after modifying the pipeline
- Fast comparison of different parameter settings

## Note

For formal evaluation against expert annotations, use `Compute_Real_Metrics.ipynb` instead. This notebook uses proxy ground truth derived from Channel 3, not expert-drawn masks.

## Runtime

Approximately 2-3 minutes for 4 timepoints.


In [1]:
# Cell 1: Imports
import numpy as np
import tifffile as tiff
from pathlib import Path
from skimage import filters, morphology
from scipy import ndimage as ndi
import pandas as pd

print("Libraries loaded!")


Libraries loaded!


In [2]:
# Cell 2: Configuration - only 4 timepoints for speed
TEST_TIMEPOINTS = [30, 100, 140, 190]
Z_SLICE = 188

MASK_DIR = Path("mask_santi")
RAW_DIR = Path(".")

print(f"Will test timepoints: {TEST_TIMEPOINTS}")
print(f"Using Z slice: {Z_SLICE}")


Will test timepoints: [30, 100, 140, 190]
Using Z slice: 188


In [3]:
# Cell 3: Helper functions
def dice_score(pred, gt):
    pred = pred.astype(bool)
    gt = gt.astype(bool)
    intersection = np.logical_and(pred, gt).sum()
    total = pred.sum() + gt.sum()
    if total == 0:
        return 1.0
    return 2.0 * intersection / total

def iou_score(pred, gt):
    pred = pred.astype(bool)
    gt = gt.astype(bool)
    intersection = np.logical_and(pred, gt).sum()
    union = np.logical_or(pred, gt).sum()
    if union == 0:
        return 1.0
    return intersection / union

def otsu_baseline(raw_slice):
    v = raw_slice.astype(np.float32)
    v = (v - v.min()) / (v.max() - v.min() + 1e-6)
    thresh = filters.threshold_otsu(v)
    mask = v > thresh
    mask = morphology.binary_closing(mask, morphology.disk(5))
    mask = ndi.binary_fill_holes(mask)
    return mask

def hysteresis_baseline(raw_slice):
    v = raw_slice.astype(np.float32)
    v = (v - v.min()) / (v.max() - v.min() + 1e-6)
    hi = np.percentile(v, 90)
    lo = np.percentile(v, 70)
    mask = filters.apply_hysteresis_threshold(v, lo, hi)
    mask = morphology.binary_closing(mask, morphology.disk(5))
    mask = ndi.binary_fill_holes(mask)
    return mask

def channel3_proxy_gt(raw_slice):
    v = raw_slice.astype(np.float32)
    v = (v - v.min()) / (v.max() - v.min() + 1e-6)
    thresh = np.percentile(v[v > 0], 75)
    mask = v > thresh
    mask = morphology.binary_closing(mask, morphology.disk(7))
    mask = ndi.binary_fill_holes(mask)
    labeled = morphology.label(mask)
    if labeled.max() > 0:
        counts = np.bincount(labeled.ravel())
        counts[0] = 0
        mask = (labeled == np.argmax(counts))
    return mask

print("Functions defined!")


Functions defined!


In [4]:
# Cell 4: Run evaluation
results = []

for t in TEST_TIMEPOINTS:
    print(f"\nProcessing t={t:04d}...", end=" ")
    
    raw_path = RAW_DIR / f"t{t:04d}_Channel 3.tif"
    if not raw_path.exists():
        print(f"Raw not found: {raw_path}")
        continue
    raw = tiff.imread(str(raw_path))
    raw_slice = raw[Z_SLICE]
    
    mask_path = MASK_DIR / f"mask_t{t:04d}_Channel 3.tif"
    if not mask_path.exists():
        print(f"Mask not found: {mask_path}")
        continue
    our_mask = tiff.imread(str(mask_path))
    our_slice = our_mask[Z_SLICE] > 0
    
    gt_slice = channel3_proxy_gt(raw_slice)
    otsu_slice = otsu_baseline(raw_slice)
    hyst_slice = hysteresis_baseline(raw_slice)
    
    results.append({"timepoint": t, "method": "Otsu", "dice": dice_score(otsu_slice, gt_slice), "iou": iou_score(otsu_slice, gt_slice)})
    results.append({"timepoint": t, "method": "Hysteresis", "dice": dice_score(hyst_slice, gt_slice), "iou": iou_score(hyst_slice, gt_slice)})
    results.append({"timepoint": t, "method": "Ours (RF)", "dice": dice_score(our_slice, gt_slice), "iou": iou_score(our_slice, gt_slice)})
    
    print("OK")

print("\n" + "="*60)
print("DONE!")



Processing t=0030... OK

Processing t=0100... OK

Processing t=0140... OK

Processing t=0190... OK

DONE!


In [5]:
# Cell 5: Summary and LaTeX table
df = pd.DataFrame(results)
print("\n" + "="*60)
print("RESULTS BY TIMEPOINT")
print("="*60)
print(df.to_string(index=False))

print("\n" + "="*60)
print("SUMMARY BY METHOD (mean +/- std)")
print("="*60)
summary = df.groupby("method").agg({"dice": ["mean", "std"], "iou": ["mean", "std"]}).round(3)
print(summary)

print("\n" + "="*60)
print("LATEX TABLE (copy this to your report)")
print("="*60)
for m in ["Otsu", "Hysteresis", "Ours (RF)"]:
    sub = df[df["method"] == m]
    d_mean, d_std = sub["dice"].mean(), sub["dice"].std()
    i_mean, i_std = sub["iou"].mean(), sub["iou"].std()
    print(f"{m} & {d_mean:.3f} $\\pm$ {d_std:.3f} & {i_mean:.3f} $\\pm$ {i_std:.3f} \\\\")

df.to_csv("quick_metrics_results.csv", index=False)
print("\nSaved to quick_metrics_results.csv")



RESULTS BY TIMEPOINT
 timepoint     method     dice      iou
        30       Otsu 0.187603 0.103511
        30 Hysteresis 0.561517 0.390353
        30  Ours (RF) 0.072946 0.037854
       100       Otsu 0.999445 0.998890
       100 Hysteresis 0.567162 0.395832
       100  Ours (RF) 0.041032 0.020946
       140       Otsu 0.808939 0.679175
       140 Hysteresis 0.543091 0.372769
       140  Ours (RF) 0.039219 0.020002
       190       Otsu 0.999974 0.999948
       190 Hysteresis 0.874385 0.776807
       190  Ours (RF) 0.025600 0.012966

SUMMARY BY METHOD (mean +/- std)
             dice           iou       
             mean    std   mean    std
method                                
Hysteresis  0.637  0.159  0.484  0.195
Otsu        0.749  0.385  0.695  0.422
Ours (RF)   0.045  0.020  0.023  0.011

LATEX TABLE (copy this to your report)
Otsu & 0.749 $\pm$ 0.385 & 0.695 $\pm$ 0.422 \\
Hysteresis & 0.637 $\pm$ 0.159 & 0.484 $\pm$ 0.195 \\
Ours (RF) & 0.045 $\pm$ 0.020 & 0.023 $\pm$ 0.01